# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://www.myjoyonline.com/")
huggingface.links

['/',
 '/',
 'https://www.myjoyonline.com/news/',
 'https://www.myjoyonline.com/arts-culture/',
 'https://www.myjoyonline.com/business/',
 'https://www.myjoyonline.com/sports/',
 'https://www.myjoyonline.com/opinion/',
 'https://www.myjoyonline.com/research/',
 '/live',
 'https://www.myjoyonline.com/media/',
 'https://myjoyonline.com/election',
 '/',
 '/',
 'https://www.myjoyonline.com/news/',
 'https://www.myjoyonline.com/arts-culture/',
 'https://www.myjoyonline.com/business/',
 'https://www.myjoyonline.com/sports/',
 'https://www.myjoyonline.com/opinion/',
 'https://www.myjoyonline.com/research/',
 '/live',
 'https://www.myjoyonline.com/media/',
 'https://myjoyonline.com/election',
 'https://www.youtube.com/user/myjoyonlinetube?sub_confirmation=1',
 'https://web.facebook.com/JoyNewsOnTV/',
 'https://twitter.com/Joy997FM',
 '#',
 '/category/top-story',
 'https://www.myjoyonline.com/raymond-senyo-amezado-name-overall-winner-at-2025-joynews-impact-makers-awards/',
 'https://www.myjoyon

In [12]:
get_links("https://www.myjoyonline.com/")

{'links': [{'type': 'home page', 'url': 'https://www.myjoyonline.com/'},
  {'type': 'company page', 'url': 'https://www.multimediaghana.com'},
  {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'},
  {'type': 'careers page', 'url': 'https://www.myjoyonline.com/careers'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://www.myjoyonline.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.multimediaghana.com'}, {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'}, {'type': 'careers page', 'url': 'https://www.myjoyonline.com/careers'}, {'type': 'news page', 'url': 'https://www.myjoyonline.com/news/'}, {'type': 'media page', 'url': 'https://www.myjoyonline.com/media/'}]}
Landing page:
Webpage Title:
MyJoyOnline - Ghana News | Ghana's most comprehensive website. Independent, Fearless and Credible journalism
Webpage Contents:
Home
News
Arts and Culture
Business
Sports
Opinion
Research
Live TV/Radio
Media
Elections
Dark Mode
Dark mode
Home
News
Arts and Culture
Business
Sports
Opinion
Research
Live TV/Radio
Media
Elections
Forgot Password
Create Account
https://www.myjoyonline.com/-------https://www.myjoyonline.com/
Receive news updates on the go.
Select one or more categories
News
Business
Sports
Opinion
Entertainment
Breaking News
I want to receive news:
Daily
Weekly
Monthly
Daily: 0.30

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("Multimedia Ghana", "https://www.myjoyonline.com/")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.myjoyonline.com'}, {'type': 'careers page', 'url': 'https://www.multimediaghana.com'}, {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'}]}


"You are looking at a company called: Multimedia Ghana\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nMyJoyOnline - Ghana News | Ghana's most comprehensive website. Independent, Fearless and Credible journalism\nWebpage Contents:\nHome\nNews\nArts and Culture\nBusiness\nSports\nOpinion\nResearch\nLive TV/Radio\nMedia\nElections\nDark Mode\nDark mode\nHome\nNews\nArts and Culture\nBusiness\nSports\nOpinion\nResearch\nLive TV/Radio\nMedia\nElections\nForgot Password\nCreate Account\nhttps://www.myjoyonline.com/-------https://www.myjoyonline.com/\nReceive news updates on the go.\nSelect one or more categories\nNews\nBusiness\nSports\nOpinion\nEntertainment\nBreaking News\nI want to receive news:\nDaily\nWeekly\nMonthly\nDaily: 0.30p | Weekly: GHS\r\n                            2 | Monthly: GHs 9\nTop story\nRaymond Senyo Amezado named overall winner at 2025 JoyN

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Multimedia Ghana", "https://www.myjoyonline.com/")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.myjoyonline.com/'}, {'type': 'company page', 'url': 'https://www.multimediaghana.com'}, {'type': 'careers page', 'url': 'https://www.myjoyonline.com/careers'}, {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'}]}


# Multimedia Ghana Brochure

---

## Welcome to Multimedia Ghana

**Your Premier Source for Independent and Fearless Journalism**

At Multimedia Ghana, we pride ourselves on being Ghana's most comprehensive news website, delivering credible journalism that empowers our audience. With a diverse range of content covering **News**, **Arts**, **Culture**, **Business**, **Sports**, **Opinion**, and more, we are the leading media outlet that brings you closer to the stories that matter.

---

## Our Offerings

- **Real-Time News Updates**: Stay informed with breaking news across various sectors including politics, business, and sports.
- **Live TV and Radio**: Tune into our live broadcasts for an engaging media experience.
- **Diverse Multimedia Content**: Access our podcasts, videos, and in-depth articles tailored to cater to a variety of interests.
- **Special Features**: Participate in our newsletters and receive curated content based on your preferences, delivered daily, weekly, or monthly.

---

## Company Culture

At Multimedia Ghana, we foster an environment that values **independence**, **integrity**, and **creativity**. Our team is motivated by the passion to inform and inspire, and we embrace innovations that enhance our storytelling capabilities. Our culture encourages **collaboration**, **open communication**, and a commitment to **excellence**. We believe that a diverse workforce is essential to reflecting the breadth of our audience and enhancing the quality of our journalism.

---

## Our Customers

Our diverse audience ranges from everyday Ghanaians seeking reliable news to industry professionals and investors looking for market insights. We engage with our community through various platforms, ensuring that we listen to their needs and deliver content that resonates strongly. Our commitment to serving the public ensures that we prioritize accuracy and timeliness in all our reporting.

---

## Careers and Job Opportunities

Join our dynamic team at Multimedia Ghana! We are always on the lookout for passionate individuals who share our commitment to quality journalism. Whether you are a seasoned journalist, a digital media expert, or someone interested in joining the media industry, we offer a variety of career paths suited to diverse talents.

### Why Work with Us?

- **Innovative Environment**: Work in a setting that embraces creativity and forward-thinking.
- **Professional Growth**: Access training and development programs designed to enhance your skills.
- **Impactful Work**: Contribute to reliable journalism that shapes public discourse and influences positive change.

---

## Get in Touch

To learn more about our offerings or to explore career opportunities, visit our website at [MyJoyOnline](https://www.myjoyonline.com/) or contact us directly.

---

**Multimedia Ghana** – Empowering citizens through independent news and information. Join us in making a difference today!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("Multimedia Ghana", "https://www.myjoyonline.com/")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.myjoyonline.com/'}, {'type': 'about page', 'url': 'https://www.multimediaghana.com'}, {'type': 'careers page', 'url': 'https://myjoyonline.com/jobs'}, {'type': 'news page', 'url': 'https://www.myjoyonline.com/news/'}, {'type': 'about page', 'url': 'https://myjoyonline.com/about'}, {'type': 'media page', 'url': 'https://www.myjoyonline.com/media/'}, {'type': 'elections page', 'url': 'https://myjoyonline.com/election'}, {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'}]}


# Multimedia Ghana Brochure

---

## Company Overview
**Multimedia Ghana** is a leading media organization in Ghana, recognized for its independent and credible journalism. We are dedicated to providing the most comprehensive news coverage across multiple platforms, including digital, television, and radio. Our flagship platform, [MyJoyOnline](https://www.myjoyonline.com/), delivers timely news updates in categories such as Business, Sports, Arts and Culture, and much more.

---

## Our Commitment
At Multimedia Ghana, we pride ourselves on our **fearless journalism** that serves as a voice for the Ghanaian public. Our mission is to inform, educate, and engage audiences, fostering an inclusive public discourse. We believe in the power of storytelling to inspire change and empower our community.

---

## Audience
Our media channels cater to a diverse audience, including:
- **General public** seeking in-depth analysis and updates on current affairs.
- **Business professionals** looking for economic news and insights.
- **Sports enthusiasts** following Ghanaian athletes and teams.
- **Cultural consumers** interested in the dynamic arts scene of Ghana.

We are committed to providing news that matters and keeping our audience informed about the latest happenings in Ghana and beyond.

---

## Company Culture
### Empowering Environment
We foster an environment where creativity and innovation thrive. Our team is encouraged to express their ideas and pursue new ways of delivering content. We believe that diversity in thought leads to comprehensive journalism that better serves our audience.

### Collaboration & Growth
At Multimedia Ghana, teamwork is integral to our success. Employees work collaboratively across departments to produce high-quality content. We also place great emphasis on professional development, offering training and growth opportunities to all team members.

---

## Careers at Multimedia Ghana
We are continuously on the lookout for talented individuals to join our dynamic team. Open positions range from editorial roles, marketing, technical support, to business development. We value your unique experiences and perspectives.

**Why Join Us?**
- Be part of a recognized and respected media brand in Ghana.
- Contribute to impactful storytelling that shapes public opinion.
- Enjoy a collaborative and inclusive workplace culture.
- Engage in continuous learning and professional growth.

---

## Connect With Us
Stay informed and be part of our journey. Follow us online for the latest news updates, or visit [MyJoyOnline](https://www.myjoyonline.com/) to explore our content.

---

**Multimedia Ghana – Informing, Inspiring, and Activating Change.**

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}



# Hugging Face: Building the Future of AI

## Who We Are
Hugging Face is a leading platform for the machine learning community, dedicated to fostering collaboration in the development of AI models, datasets, and applications. Our mission is to democratize AI by creating an open-source foundation that allows individuals and organizations to contribute, share, and innovate together.

## Our Offerings
### Explore
- **1M+ Models:** Access a vast array of machine learning models covering various modalities including text, image, video, audio, and 3D.
- **250k+ Datasets:** Browse through extensive datasets to enhance your ML projects.
- **AI Applications:** Discover and create applications using our extensive library of resources.

### Collaboration
We prioritize community engagement, offering unlimited public models, datasets, and applications for hosting and collaboration. Join a vibrant community of AI enthusiasts, researchers, and developers.

### Enterprise Solutions
For businesses seeking to harness AI, we provide enterprise-grade solutions that ensure security, scalability, and ongoing support. Our offerings include:
- **Compute Power:** Effortlessly deploy applications on dedicated GPU servers, starting at $0.60/hour.
- **Team Support:** Enterprise plans starting at $20/user/month, designed for larger organizations with additional features.

## Our Customers
Hugging Face is trusted by over 50,000 organizations, including industry leaders like:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
Each of these companies relies on our platform to enhance their AI capabilities and drive innovation.

## Community & Culture
At Hugging Face, we encourage a culture of openness, collaboration, and mutual support. We believe in building connections within the AI community through forums, newsletters, and events. Our team is passionate about creating tools that serve researchers, developers, and enthusiasts alike.

### Join Us
We are always looking for talented and passionate individuals to join our team. If you're interested in embarking on a journey to shape the future of AI, check out our [careers page](https://huggingface.co/jobs) to find opportunities at Hugging Face.

## Connect with Us
Stay updated with our latest developments:
- **[Twitter](https://twitter.com/huggingface)**
- **[LinkedIn](https://linkedin.com/company/huggingface)**
- **[Discord](https://discord.com/invite/huggingface)**

---

*Join the AI revolution at Hugging Face, where the community builds the future of machine learning together!*



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>